# Collect all settled locations

In [1]:
import geopandas as gpd
import pandas as pd
import geopy
import osmnx as ox
iq_locations = gpd.read_file('data/settled_locations/irq_pplp_ocha_20140722.shp')
location_csv = pd.DataFrame()
location_csv['name'] = iq_locations['PNameEn']
location_csv['region'] = iq_locations['A1NameEn']
location_csv['country'] = 'Iraq'
location_csv['lat'] = iq_locations['Latitude']
location_csv['lon'] = iq_locations['Longitude']
location_csv['location_type'] = 'town'
location_csv['conflict_date'] = 0
location_csv['pop/cap'] = 0

# Get population numbers

In [2]:
pop87 = pd.read_csv('data/undata_1987_census/UNdata_Export_20180505_224541956.csv')
pop87 = pop87[:-2] # drop nans
pop87

,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Iraq,1987,Total,Both Sexes,Adhamiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,464151.0,NaN
1,Iraq,1987,Total,Both Sexes,Amara,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,208797.0,NaN
2,Iraq,1987,Total,Both Sexes,BAGHDAD,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,3841268.0,1.0
3,Iraq,1987,Total,Both Sexes,Basra,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,406296.0,NaN
4,Iraq,1987,Total,Both Sexes,Diwaniya,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,196519.0,NaN
5,Iraq,1987,Total,Both Sexes,Erbil,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,485968.0,NaN
6,Iraq,1987,Total,Both Sexes,Hilla,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,268834.0,NaN
7,Iraq,1987,Total,Both Sexes,Kadhimain,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,521444.0,NaN
8,Iraq,1987,Total,Both Sexes,Karradah Sharqiyah,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,235554.0,NaN
9,Iraq,1987,Total,Both Sexes,Kerbala,City proper,Census - de facto - complete tabulation,"Final figure, complete",1987.0,296705.0,NaN


In [97]:
replace_dict

{'Adhamiya': 0    464151.0
 Name: Value, dtype: float64}

In [163]:
iq_pop_bydist = gpd.read_file('data/Iraq_Districts/iraq_districts.shp')
iq_pop_bydist['num_settlements'] = 0
iq_pop_bydist['pop87'] = 0
iq_pop_bydist['pop_2015'] = 0
iq_pop_bydist['pred_pop'] = 0

replace_dict = {}
# this had to be done by hand to check spellings and varied Eng-Arabic transliteration
replace_dict['Adhamiya'] = pop87.Value[pop87.City== 'Adhamiyah']
replace_dict['Amara'] = pop87.Value[pop87.City== 'Amara']
# districts not given unique counts in 87: 
replace_dict['Tarmia'] = pop87.Value[pop87.City== 'Baghdad'] / 
replace_dict['Khadamiya'] = pop87.Value[pop87.City== 'Kadhimain']
replace_dict[''] = pop87.Value[pop87.City== 'Karradah Sharqiyah']

SyntaxError: invalid syntax (<ipython-input-163-04a59332f1da>, line 12)

# Above was taking too long, so just going to use worldpop estimates,  crs matches for locations and worldpop raster

In [ ]:
import rasterio as rio
pops = rio.open('data/IRQ_ppp_v2b_2015_UNadj/IRQ_ppp_v2b_2015_UNadj.tif')

import matplotlib.pyplot as plt
import numpy as np

In [157]:
pops_dat = pops.read().squeeze()
type(pops_dat)
#plt.imshow(pops_dat, cmap = 'hot')

numpy.ndarray

# now getting the routes between all locations
start with just doing great circle distance, then maybe move to routing values

In [15]:
dist_iq_locations = iq_locations.copy().sample(200)
dist_iq_locations.head()

,PCode,PNameEn,PNameAr,Longitude,Latitude,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry
14002,IQ-P13967,Bani Maran,,44.370000,36.040000,1106013.0,Bani Maran,IQ-G11,Erbil,IQ-D065,Koisnjaq,POINT (44.37 36.04)
12079,IQ-P12047,Muhammad Chawsh,,45.400000,33.700000,1005482.0,Muhammad Chawsh,IQ-G10,Diyala,IQ-D057,Baladrooz,POINT (45.4 33.7)
7334,IQ-P07317,Hudaymah,,44.630000,32.250000,603420.0,Hudaymah,IQ-G06,Babylon,IQ-D034,Hashimiya,POINT (44.63 32.25)
9380,IQ-P10059,Karuka,,43.770000,37.110000,804587.0,Karuka,IQ-G08,Dahuk,IQ-D048,Amedi,POINT (43.77 37.11)
21050,IQ-P21012,Qaryat Misqlat Alia,,43.042892,36.594774,1510950.0,Qaryat Misqlat Alia,IQ-G15,Ninewa,IQ-D091,Tilkaif,POINT (43.042892 36.594774)


In [248]:
dist_df = pd.DataFrame()
for i in dist_iq_locations.itertuples():
    dist_df[i.PNameEn] = dist_iq_locations.geometry.distance(i.geometry)
dist_df.index = dist_df.columns
dist_df.to_csv('distance_matrix.csv')

KeyboardInterrupt: 

In [34]:
trips = set()
count = 0
with open("data/adj_mat_test.csv", "w") as f:
    f.write(",".join(["name1", "name2", "distance\n"]))
    for i in dist_iq_locations.itertuples():
        dists = dist_iq_locations.geometry.distance(i.geometry)
        tups = zip(dist_iq_locations.PNameEn, dists)
        for destination, time in tups:
            key = (i.PNameEn, destination)
            if key not in trips:
                trips.add(key)
                trips.add((key[1], key[0]))
                f.write(",".join([key[0], key[1], str(time)]) + "\n")
                if key[0] != key[1]:
                    f.write(",".join([key[1], key[0], str(time)]) + "\n")
dist_iq_locations.to_csv('data/iq_dist_locations_test.csv')

# conflict locations for location_csv

In [107]:
conflict_loc = pd.read_csv('https://data.humdata.org/hxlproxy/data.csv?url=https%3A%2F%2Fapi.acleddata.com%2Facled%2Fread.csv%3Flimit%3D0%26iso%3D368&name=ACLEDHXL&tagger-match-all=on&tagger-02-header=iso&tagger-02-tag=%23country%2Bcode&tagger-03-header=event_id_cnty&tagger-03-tag=%23event%2Bcode&tagger-05-header=event_date&tagger-05-tag=%23date%2Boccurred+&tagger-08-header=event_type&tagger-08-tag=%23event%2Btype&tagger-09-header=actor1&tagger-09-tag=%23group%2Bname%2Bfirst&tagger-10-header=assoc_actor_1&tagger-10-tag=%23group%2Bname%2Bfirst%2Bassoc&tagger-12-header=actor2&tagger-12-tag=%23group%2Bname%2Bsecond&tagger-13-header=assoc_actor_2&tagger-13-tag=%23group%2Bname%2Bsecond%2Bassoc&tagger-16-header=region&tagger-16-tag=%23region%2Bname&tagger-17-header=country&tagger-17-tag=%23country%2Bname&tagger-18-header=admin1&tagger-18-tag=%23adm1%2Bname&tagger-19-header=admin2&tagger-19-tag=%23adm2%2Bname&tagger-20-header=admin3&tagger-20-tag=%23adm3%2Bname&tagger-21-header=location&tagger-21-tag=%23loc%2Bname&tagger-22-header=latitude&tagger-22-tag=%23geo%2Blat&tagger-23-header=longitude&tagger-23-tag=%23geo%2Blon&tagger-25-header=source&tagger-25-tag=%23meta%2Bsource&tagger-27-header=notes&tagger-27-tag=%23description&tagger-28-header=fatalities&tagger-28-tag=%23affected%2Bkilled&header-row=1')
conflict_loc = conflict_loc.iloc[1:]
conflict_loc.to_csv("data/acled_conflict_locations.csv")


In [164]:
from shapely.geometry import Point
conflict_loc_geo = gpd.GeoDataFrame(conflict_loc)
points = [Point(float(j), float(i)) for i, j in zip(conflict_loc.latitude, conflict_loc.longitude)]
geometries_acled = gpd.GeoSeries(points).buffer(0.1)
# align geometries - can't know what geometry acled was using...
acled_locations = gpd.GeoDataFrame()
acled_locations['geometry'] = geometries_acled
acled_locations.crs = iq_locations.crs
acled_locations.to_crs(iq_locations.crs)
acled_locations['date'] = conflict_loc['event_date']
acled_locations.to_file('data/acled_conflict_geometries.shp')

In [166]:
acled_temp = gpd.sjoin(iq_locations, acled_locations, 'inner')
acled_temp

,PCode,PNameEn,PNameAr,Longitude,Latitude,PCodeAlt1,PNameAlt1,A1Code,A1NameEn,A2Code,A2NameEn,geometry,index_right,date
0,IQ-P00001,Al 'Bor (Shishan),حي العبور,41.977818,34.369352,110013.0,Al 'bor (Shishan),IQ-G01,Anbar,IQ-D001,Ana,POINT (41.977818 34.369352),2902,2017-08-18
2,IQ-P00003,Al Tadamon,حي التضامن,41.972490,34.367327,110017.0,Al Tadamon,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.97249 34.367327),2902,2017-08-18
3,IQ-P00004,Al-A'Wany Village,قرية العواني,41.987483,34.373356,110318.0,Al-A'wany Village,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.987483 34.373356),2902,2017-08-18
8,IQ-P00009,Hay al Nasir,حي النصر,41.990583,34.366033,110015.0,Hay Al Nasir,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.990583 34.366033),2902,2017-08-18
9,IQ-P00010,Hay al Qadissiyah,حي القادسية,41.980873,34.376623,110016.0,Hay Al Qadissiyah,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.980873 34.376623),2902,2017-08-18
10,IQ-P00011,Hay al Salam,حي السلام,41.980244,34.364500,110018.0,Hay Al Salam,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.980244 34.3645),2902,2017-08-18
11,IQ-P00012,Hay al Tammem,حي التأميم,41.991556,34.369383,110014.0,Hay Al Tammem,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.991556 34.369383),2902,2017-08-18
12,IQ-P00013,Hay al Zera'I,حي الزراعي,41.977818,34.369352,110242.0,Hay Al Zera'I,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.977818 34.369352),2902,2017-08-18
17,IQ-P00018,Rayhana,مجمع الريحانه,42.029125,34.340626,110012.0,Rayhana,IQ-G01,Anbar,IQ-D001,Ana,POINT (42.029125 34.340626),2902,2017-08-18
18,IQ-P00019,Yarmouk,حي اليرموك,41.988392,34.370033,109067.0,Yarmouk,IQ-G01,Anbar,IQ-D001,Ana,POINT (41.988392 34.370033),2902,2017-08-18
